In [78]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# Toeknising the titles 
We will now decapatilise and tokenise(segment) the titles so that we can process them

In [79]:
#importing df of story rows rather that comments 
df = pd.read_csv('output.csv')


In [80]:
df['title'].head

<bound method NDFrame.head of 0          "What May Happen in the Next Hundred Years", f...
1               Getting Started with JavaScript Unit Testing
2          Armstrong, the Django-based and open-source ne...
3                       Why Web Reviewers Make Up Bad Things
4          You Weren't Meant to Have a Boss: The Cliff Notes
                                 ...                        
5351743    Scrapling: Fast, Adaptive Web Scraping for Python
5351744                         Monkeys Predict US Elections
5351745                                                  NaN
5351746                                                  NaN
5351747                                                  NaN
Name: title, Length: 5351748, dtype: object>

In [81]:
df = df.dropna(subset=['title'])

In [82]:
df = df.head(2500)

In [83]:
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Set seed for consistent language detection results
DetectorFactory.seed = 0

# Function to check if the title is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False  # If detection fails, we assume it's not English

# Filter rows where the title is in English
df['title'] = df['title'].str.lower()
df = df[df['title'].apply(is_english)]

# Display the modified DataFrame
print(df)

           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [84]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import mapply

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

# Initialize the stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function to apply stemming and lemmatization
def process_text(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    
    # Stemming
    stemmed = [stemmer.stem(word) for word in words]
    
    # Lemmatization
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(stemmed), ' '.join(lemmatized)

# Apply the function to the 'title' column
df['stemmed'], df['lemmatized'] = zip(*df['title'].apply(process_text))

# Display the modified DataFrame
print(df)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omare\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\omare\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\omare\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [85]:
import pandas as pd
import re

# Function to replace punctuation with special tokens
def preprocess_text(text):
    # Replace specific punctuation with special tokens
    text = text.replace(',', '<COMMA>')  # Replace commas
    text = text.replace('.', '<FULLSTOP>')  # Replace full stops
    text = text.replace('!', '<EXCLAMATION>')  # Replace exclamations
    return text

# Tokenize the text (split into individual words and special tokens)
def tokenize_text(text):
    # Tokenize by splitting on whitespace and keeping punctuation tokens
    return re.findall(r'\S+|<COMMA>|<FULLSTOP>|<EXCLAMATION>', text)

# Preprocess and tokenize the 'title' column
df['title'] = df['title'].apply(preprocess_text)
df['tokens'] = df['title'].apply(tokenize_text)

# Create a vocabulary (a dictionary mapping tokens to unique IDs)
def create_vocabulary(tokens_series):
    # Flatten the list of token lists into a single list
    all_tokens = [token for tokens in tokens_series for token in tokens]
    # Create a unique token list and assign an ID to each
    vocabulary = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
    return vocabulary

# Get vocabulary from the tokens in the DataFrame
vocab = create_vocabulary(df['tokens'])

# Function to convert tokens to token IDs using the vocabulary
def tokens_to_ids(tokens, vocab):
    return [vocab[token] for token in tokens]

# Apply the function to convert tokens to token IDs
df['token_ids'] = df['tokens'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

# Display the DataFrame with token IDs
print(df)


           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [86]:
import pandas as pd
import re


# Function to replace all punctuation with special tokens
def preprocess_text(text):
    # Replace various punctuation with special tokens
    replacements = {
        ',': '<COMMA>',
        '.': '<FULLSTOP>',
        '!': '<EXCLAMATION>',
        '?': '<QUESTION>',
        ';': '<SEMICOLON>',
        ':': '<COLON>',
        '(': '<OPENPAREN>',
        ')': '<CLOSEPAREN>',
        '[': '<OPENBRACKET>',
        ']': '<CLOSEBRACKET>',
        '{': '<OPENBRACE>',
        '}': '<CLOSEBRACE>',
        '\'': '<QUOTE>',
        '\"': '<DOUBLEQUOTE>',
        '-': '<DASH>',
        '_': '<UNDERLINE>',
        '…': '<ELLIPSIS>'  # Ellipsis
    }
    
    for punctuation, token in replacements.items():
        text = text.replace(punctuation, token)  # Replace each punctuation
    return text

# Tokenize the text (split into individual words and special tokens)
def tokenize_text(text):
    # Tokenize by splitting on whitespace and keeping punctuation tokens
    return re.findall(r'\S+|<COMMA>|<FULLSTOP>|<EXCLAMATION>|<QUESTION>|<SEMICOLON>|<COLON>|<OPENPAREN>|<CLOSEPAREN>|<OPENBRACKET>|<CLOSEBRACKET>|<OPENBRACE>|<CLOSEBRACE>|<QUOTE>|<DOUBLEQUOTE>|<DASH>|<UNDERLINE>|<ELLIPSIS>', text)

# Preprocess and tokenize the 'title' column
df['title'] = df['title'].apply(preprocess_text)
df['tokens'] = df['title'].apply(tokenize_text)

# Create a vocabulary (a dictionary mapping tokens to unique IDs)
def create_vocabulary(tokens_series):
    # Flatten the list of token lists into a single list
    all_tokens = [token for tokens in tokens_series for token in tokens]
    # Create a unique token list and assign an ID to each
    vocabulary = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
    return vocabulary

# Get vocabulary from the tokens in the DataFrame
vocab = create_vocabulary(df['tokens'])

# Function to convert tokens to token IDs using the vocabulary
def tokens_to_ids(tokens, vocab):
    return [vocab[token] for token in tokens]

# Apply the function to convert tokens to token IDs
df['token_ids'] = df['tokens'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

# Display the DataFrame with token IDs
print(df)


           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [87]:

#import pandas as pd
#import sentencepiece as spm


# Step 1: Preprocess the text (optional)
# def preprocess_text(text):
#     # Replace specific punctuation with special tokens
#     text = text.replace(',', '<COMMA>')  # Replace commas
#     text = text.replace('.', '<FULLSTOP>')  # Replace full stops
#     text = text.replace('!', '<EXCLAMATION>')  # Replace exclamations
#     return text

# # Preprocess the 'title' column
# df['title'] = df['title'].apply(preprocess_text)

# # Step 2: Save titles to a file for training SentencePiece
# with open('titles.txt', 'w') as f:
#     for title in df['title']:
#         f.write(title + '\n')

# # Step 3: Train a SentencePiece model
# # Increase vocab_size to at least 100 to avoid the error
# spm.SentencePieceTrainer.train('--input=titles.txt --model_prefix=m --vocab_size=100 --character_coverage=1.0')

# # Step 4: Load the trained SentencePiece model
# sp = spm.SentencePieceProcessor(model_file='m.model')

# # Tokenize the 'title' column using SentencePiece
# df['tokens'] = df['title'].apply(lambda x: sp.encode(x, out_type=str))

# # Create a vocabulary from the tokens
# def create_vocabulary(tokens_series):
#     all_tokens = [token for tokens in tokens_series for token in tokens]
#     vocabulary = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
#     return vocabulary

# # Get vocabulary from the tokens in the DataFrame
# vocab = create_vocabulary(df['tokens'])

# # Function to convert tokens to token IDs using the vocabulary
# def tokens_to_ids(tokens, vocab):
#     return [vocab[token] for token in tokens]

# # Apply the function to convert tokens to token IDs
# df['token_ids'] = df['tokens'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

# # Display the DataFrame with tokens and token IDs
# print(df)
